# Install library

In [1]:
!pip install rouge-score

  Preparing metadata (setup.py) ... done
  Created wheel for rouge-score: filename=rouge_score-0.1.2-py3-none-any.whl size=24935 sha256=712ee134d641e5c715ebd25db7c06f32317f3acca28c927871411dfb3b8c6a13
  Stored in directory: /root/.cache/pip/wheels/5f/dd/89/461065a73be61a532ff8599a28e9beef17985c9e9c31e541b4
Successfully built rouge-score


# Import Library

In [6]:
import pandas as pd
from torch.utils.data import Dataset, DataLoader
import torch
from transformers import AutoTokenizer, AutoModelForSequenceClassification, AdamW
from tqdm.notebook import tqdm
import time

# Download and preprocess data

## Download data

In [7]:
import gdown
"""https://drive.google.com/file/d/1YgEEMPTKUSFnb5GyeiIb1PIej65_XAFD/view?usp=sharing"""

# Thay thế bằng ID của tệp ZIP trên Google Drive
file_id = '1YgEEMPTKUSFnb5GyeiIb1PIej65_XAFD'

# Tải xuống tệp ZIP
gdown.download(f"https://drive.google.com/uc?id={file_id}", output="/kaggle/working/data.zip", quiet=False)


Downloading...
From (original): https://drive.google.com/uc?id=1YgEEMPTKUSFnb5GyeiIb1PIej65_XAFD
From (redirected): https://drive.google.com/uc?id=1YgEEMPTKUSFnb5GyeiIb1PIej65_XAFD&confirm=t&uuid=dbb4afb4-36a5-4f28-ad00-989de01aafc7
To: /kaggle/working/data.zip
100%|██████████| 58.5M/58.5M [00:00<00:00, 211MB/s]


'/kaggle/working/data.zip'

In [8]:
import zipfile
import os

# Đường dẫn tới file zip mà bạn muốn giải nén
zip_file_path = '/kaggle/working/data.zip'

# Đường dẫn thư mục bạn muốn giải nén file vào
output_directory = '/kaggle/working'

# Tạo thư mục đầu ra nếu nó chưa tồn tại
os.makedirs(output_directory, exist_ok=True)

# Giải nén file
with zipfile.ZipFile(zip_file_path, 'r') as zip_ref:
    zip_ref.extractall(output_directory)

print(f"Đã giải nén file zip tới {output_directory}")


Đã giải nén file zip tới /kaggle/working


## Preprocessing Data

In [9]:
def aggregated_data():
    languages = ["c#", "go", "html", "java", "javascript", "php", "python", "ruby"]
    train = pd.DataFrame({"prefix": [], "input_text": [], "target_text": []})
    test = pd.DataFrame({"prefix": [], "input_text": [], "target_text": []})
    val = pd.DataFrame({"prefix": [], "input_text": [], "target_text": []})
    i = 0
    for language in languages:
        # Đọc dữ liệu huấn luyện, kiểm tra và xác nhận
        language_train = pd.read_csv(f"/kaggle/working/datasets/{language}/train.csv")
        train = pd.concat([train, language_train], ignore_index=True).dropna()
        language_test = pd.read_csv(f"/kaggle/working/datasets/{language}/test.csv")
        test = pd.concat([test, language_test], ignore_index=True).dropna()
        
        if language != "javascript":
            language_val = pd.read_csv(f"/kaggle/working/datasets/{language}/valid.csv")
            val = pd.concat([val, language_val], ignore_index=True).dropna()
    # print(len(train))
    # print(train["input_text"][0])
    print(train.head())
    # Lưu các tệp tổng hợp
    train.to_csv("/kaggle/working/train.csv", index=False)
    test.to_csv("/kaggle/working/test.csv", index=False)
    val.to_csv("/kaggle/working/val.csv", index=False)


In [10]:
aggregated_data()

  prefix                                         input_text  \
0     c#  How to convert a Decimal <mask> <body> I want ...   
1     c#  How to convert a Decimal to a <mask> <body> I ...   
2     c#  How to convert a <mask> <body> I want to use a...   
3     c#  Filling a DataSet or a DataTable from a <mask>...   
4     c#  Filling <mask> <body> How do you expose a LINQ...   

                                         target_text  
0                                 to a Double in C#?  
1                                      Double in C#?  
2                         Decimal to a Double in C#?  
3                              LINQ query result set  
4  a DataSet or a DataTable from a LINQ query res...  


# Initialize Model

In [11]:
## Set up parameters
device = torch.device("cuda" if torch.cuda.is_available() else "cpu") 

In [12]:
from transformers import T5Tokenizer, T5ForConditionalGeneration

# Tải mô hình và tokenizer của T5 từ thư viện Transformers
model_name = 't5-base'  # Tên mô hình có thể thay đổi, ví dụ: 't5-small', 't5-large'
tokenizer = T5Tokenizer.from_pretrained(model_name)  # Tải tokenizer phù hợp với mô hình đã chọn
model = T5ForConditionalGeneration.from_pretrained(model_name)  # Tải mô hình T5 đã được huấn luyện trước
model.to(device)  # Chuyển mô hình sang thiết bị (CPU hoặc GPU) để tăng tốc xử lý


spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.39M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.21k [00:00<?, ?B/s]

You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thoroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565


model.safetensors:   0%|          | 0.00/892M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

T5ForConditionalGeneration(
  (shared): Embedding(32128, 768)
  (encoder): T5Stack(
    (embed_tokens): Embedding(32128, 768)
    (block): ModuleList(
      (0): T5Block(
        (layer): ModuleList(
          (0): T5LayerSelfAttention(
            (SelfAttention): T5Attention(
              (q): Linear(in_features=768, out_features=768, bias=False)
              (k): Linear(in_features=768, out_features=768, bias=False)
              (v): Linear(in_features=768, out_features=768, bias=False)
              (o): Linear(in_features=768, out_features=768, bias=False)
              (relative_attention_bias): Embedding(32, 12)
            )
            (layer_norm): T5LayerNorm()
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (1): T5LayerFF(
            (DenseReluDense): T5DenseActDense(
              (wi): Linear(in_features=768, out_features=3072, bias=False)
              (wo): Linear(in_features=3072, out_features=768, bias=False)
              (dropout): Dro

In [13]:
# Tạo danh sách chứa các tham số của mô hình, bao gồm tên và giá trị của từng tham số
param_optimizer = list(model.named_parameters())  

# Danh sách các tham số không áp dụng weight decay (không giảm trọng số)
no_decay = ['bias', 'LayerNorm.weight']  

optimizer_grouped_parameters = [
    {'params': [p for n, p in param_optimizer if not any(nd in n for nd in no_decay)],
    # Nhóm các tham số sẽ áp dụng weight decay (ngoại trừ bias và LayerNorm.weight)
     'weight_decay_rate': 0.01},  
    {'params': [p for n, p in param_optimizer if any(nd in n for nd in no_decay)],
    # Nhóm các tham số không áp dụng weight decay (bias và LayerNorm.weight)
     'weight_decay_rate': 0.0}  
]
# Tạo đối tượng optimizer AdamW với các tham số nhóm và thiết lập learning rate là 5e-5
optimizer = AdamW(optimizer_grouped_parameters, lr=5e-5)  

# Số lần cho phép không cải thiện trước khi dừng sớm (early stopping)
early_stopping_patience = 10  

# Số lượng beam search được sử dụng trong quá trình giải mã (decoding)
num_beams = 10  

# Độ dài tối đa của câu đầu vào
max_input_length = 512  

# Độ dài tối đa của câu đầu ra
max_output_length = 48  

# Hệ số phạt độ dài (penalty) để kiểm soát độ dài chuỗi đầu ra trong beam search
length_penalty = 1.2  

# Số lượng mẫu trong mỗi batch
batch_size = 8
# Số epoch (vòng lặp qua toàn bộ dữ liệu) cho quá trình huấn luyện
epochs = 1  


/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:591: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


## Preprocess function for Model


In [14]:
# Hàm tiền xử lý (preprocessing) một mẫu (example) từ dữ liệu
def preprocess_function(example):

    # Tạo dictionary để lưu các dữ liệu đầu vào sau khi xử lý
    model_input = {}

    # Gộp "prefix" và "input_text" để tạo thành một chuỗi văn bản duy nhất (input_text)
    input_text = example["prefix"] + " " + example["input_text"]

    model_input["input_ids"] = torch.tensor(
        tokenizer(input_text, max_length=max_input_length, truncation=True, padding="max_length").input_ids
    )
    # Tokenize input_text: chuyển văn bản đầu vào thành input IDs (số nguyên tương ứng với các từ)
    # Áp dụng các tham số:
    # - max_length: độ dài tối đa của chuỗi input (đầu vào)
    # - truncation: cắt bớt chuỗi nếu vượt quá max_length
    # - padding: điền thêm các giá trị "padding" nếu chuỗi ngắn hơn max_length
    # Kết quả là input_ids được chuyển sang Tensor

    model_input["label"] = torch.tensor(
        tokenizer(example["target_text"], max_length=max_output_length, truncation=True, padding="max_length").input_ids
    )
    # Tokenize target_text: chuyển văn bản đầu ra (target_text) thành input IDs
    # Áp dụng các tham số tương tự: max_length, truncation, và padding
    # Kết quả là label được chuyển sang Tensor

    return model_input
    # Trả về dictionary chứa input_ids và label


In [15]:
class CustomDataset(Dataset):
    """Characterizes a dataset for PyTorch"""
    def __init__(self, data):
        '''Initialization'''
        self.data = data

    def __len__(self):
        '''Denotes the total number of samples'''
        return len(self.data)

    def __getitem__(self, index):
        '''Generates one sample of data'''
        return preprocess_function(self.data[index])


In [16]:
## Building Rouge-L scoring for model 
from rouge_score import rouge_scorer  
# Import thư viện `rouge_scorer` để tính toán ROUGE-L

def evaluate_rougeL(model, data_loader):
    # Hàm đánh giá mô hình bằng cách tính điểm ROUGE-L

    model.eval()
    # Chuyển mô hình sang chế độ đánh giá (evaluation mode) để tắt dropout và các kỹ thuật không cần thiết trong giai đoạn dự đoán

    predictions, true_labels = [], []
    # Khởi tạo danh sách trống để lưu trữ các dự đoán (predictions) và nhãn thực tế (true_labels)

    with torch.no_grad():
        # Tắt tính toán gradient để tiết kiệm bộ nhớ và tăng tốc trong giai đoạn đánh giá

        tqdm_batch_iterator = tqdm(data_loader)
        # Sử dụng `tqdm` để hiển thị thanh tiến trình khi lặp qua các batch dữ liệu

        for i, batch in enumerate(tqdm_batch_iterator):
            # Lặp qua từng batch dữ liệu trong data_loader

            input_ids = batch['input_ids'].squeeze().to(device)
            # Lấy các input_ids từ batch và chuyển chúng sang thiết bị (CPU hoặc GPU)

            labels = batch['label'].squeeze().to(device)
            # Lấy các nhãn (labels) từ batch và chuyển chúng sang thiết bị

            outputs = model.generate(input_ids=input_ids, max_length=max_output_length, num_beams=num_beams, length_penalty=length_penalty)
            # Sử dụng mô hình để sinh đầu ra (outputs) từ input_ids với các tham số beam search:
            # - max_length: độ dài tối đa của chuỗi được sinh
            # - num_beams: số beam để mở rộng các lựa chọn
            # - length_penalty: hệ số phạt độ dài để kiểm soát độ dài chuỗi được sinh

            predictions.extend([tokenizer.decode(g, skip_special_tokens=True, clean_up_tokenization_spaces=True) for g in outputs])
            # Giải mã (decode) các đầu ra đã sinh (`outputs`) thành văn bản và thêm chúng vào danh sách `predictions`

            true_labels.extend([tokenizer.decode(l, skip_special_tokens=True, clean_up_tokenization_spaces=True) for l in labels])
            # Giải mã (decode) các nhãn thực tế (`labels`) thành văn bản và thêm chúng vào danh sách `true_labels`

    scorer = rouge_scorer.RougeScorer(['rougeL'], use_stemmer=True)
    # Tạo một đối tượng tính điểm ROUGE (RougeScorer) với cấu hình tính ROUGE-L và bật tính năng stemming (rút gọn từ)

    rougeL_scores = []
    # Khởi tạo danh sách trống để lưu điểm số ROUGE-L của từng cặp dự đoán và nhãn thực tế

    for pred, true_label in zip(predictions, true_labels):
        # Lặp qua từng cặp dự đoán và nhãn thực tế

        scores = scorer.score(true_label, pred)
        # Tính điểm ROUGE-L giữa nhãn thực tế (`true_label`) và dự đoán (`pred`)

        rougeL_scores.append(scores['rougeL'].fmeasure)
        # Lấy giá trị F-measure của ROUGE-L và thêm vào danh sách `rougeL_scores`

    rougeL_avg = sum(rougeL_scores) / len(rougeL_scores)
    # Tính điểm ROUGE-L trung bình (average) trên tất cả các cặp

    return rougeL_avg
    # Trả về điểm ROUGE-L trung bình


## Training function

In [17]:
def Trainer():
    # Hàm Trainer để huấn luyện mô hình với dữ liệu huấn luyện và đánh giá

    torch.manual_seed(123)
    # Thiết lập seed để đảm bảo kết quả có thể tái hiện được

    train_dataloader = DataLoader(CustomDataset(data_train), shuffle=False, batch_size=batch_size)
    # Tạo DataLoader cho tập huấn luyện, không xáo trộn dữ liệu (shuffle) và sử dụng kích thước batch được định nghĩa

    validation_dataloader = DataLoader(CustomDataset(data_val), shuffle=False, batch_size=batch_size)
    # Tạo DataLoader cho tập validation với cấu hình tương tự

    best_rougeL = float(0)
    # Biến lưu điểm ROUGE-L tốt nhất trong quá trình huấn luyện

    early_stopping_counter = 0
    # Biến đếm số lần liên tiếp mà mô hình không cải thiện (dùng cho Early Stopping)
    
    for epoch in tqdm(range(epochs), desc="Epochs"):
        # Lặp qua từng epoch, hiển thị tiến trình của các epoch bằng tqdm
        
        model.train()
        # Đưa mô hình về chế độ huấn luyện

        total_loss = 0
        # Biến để lưu tổng loss trong mỗi epoch
        
        batch_time_avg = 0.0
        # Biến để tính thời gian trung bình mỗi batch
        
        tqdm_batch_iterator = tqdm(train_dataloader)
        # Hiển thị tiến trình của từng batch dữ liệu trong DataLoader

        for i, batch in enumerate(tqdm_batch_iterator):
            # Lặp qua từng batch trong DataLoader
            
            batch_start = time.time()
            # Ghi lại thời gian bắt đầu xử lý batch
            
            input_ids = batch['input_ids'].squeeze().to(device)
            # Lấy `input_ids` từ batch và chuyển sang thiết bị (CPU/GPU)

            labels = batch['label'].squeeze().to(device)
            # Lấy `label` từ batch và chuyển sang thiết bị
            
            optimizer.zero_grad()
            # Xóa gradient trước mỗi bước cập nhật
            
            outputs = model(input_ids=input_ids, labels=labels)
            # Truyền dữ liệu qua mô hình để nhận kết quả đầu ra
            
            loss = outputs.loss
            # Lấy giá trị loss từ kết quả của mô hình
            
            loss.backward()
            # Lan truyền ngược (backpropagation) để tính gradient
            
            total_loss += loss.item()
            # Cộng dồn loss của batch hiện tại vào tổng loss
            
            optimizer.step()
            # Cập nhật các tham số của mô hình dựa trên gradient
            
            batch_time_avg += time.time() - batch_start
            # Tính thời gian xử lý cho batch hiện tại và cập nhật tổng thời gian
            
            description = "Avg. batch proc. time: {:.4f}s, loss: {:.4f}"\
                          .format(batch_time_avg/(i+1), total_loss/(i+1))
            # Định nghĩa chuỗi mô tả để hiển thị thời gian trung bình và loss trung bình
            
            tqdm_batch_iterator.set_description(description)
            # Cập nhật mô tả cho thanh tiến trình
        
        avg_loss = total_loss / len(train_dataloader)
        # Tính loss trung bình cho toàn bộ epoch
        
        print(f'Epoch: {epoch + 1}, Loss: {avg_loss}')
        # Hiển thị thông tin loss trung bình sau mỗi epoch
        
        rougeL = evaluate_rougeL(model, validation_dataloader)
        # Đánh giá mô hình trên tập validation và tính ROUGE-L

        print(f"Epoch {epoch + 1} completed. Validation ROUGE-L: {rougeL:.4f}")
        # Hiển thị điểm ROUGE-L trên tập validation cho epoch hiện tại
        
        if rougeL > best_rougeL:
            # Nếu điểm ROUGE-L hiện tại tốt hơn điểm tốt nhất trước đó
            
            best_rougeL = rougeL
            # Cập nhật điểm ROUGE-L tốt nhất
            
            model.save_pretrained(f'the_T5_model_rds')
            # Lưu mô hình vào thư mục được chỉ định
            
            print(f"Model saved at epoch {epoch + 1} with ROUGE-L: {best_rougeL:.4f}")
            # Hiển thị thông báo rằng mô hình đã được lưu
        else:
            # Nếu ROUGE-L không được cải thiện
            early_stopping_counter += 1
            # Tăng biến đếm Early Stopping
            
            if early_stopping_counter >= early_stopping_patience:
                # Nếu số lần không cải thiện vượt quá giới hạn cho phép
                
                print("Early stopping triggered.")
                # Hiển thị thông báo kích hoạt Early Stopping
                
                print(f"Training completed. Best ROUGE-L score: {best_rougeL:.4f}")
                # Hiển thị điểm ROUGE-L tốt nhất đạt được và dừng huấn luyện
                
                break
                # Thoát khỏi vòng lặp huấn luyện
        
    print(f"Training completed. Best ROUGE-L score: {best_rougeL:.4f}")
    # Hoàn tất quá trình huấn luyện và hiển thị điểm ROUGE-L tốt nhất


## Evaluation

In [18]:
def Tester():
    # Hàm Tester để đánh giá mô hình trên tập kiểm tra (test set)

    model = T5ForConditionalGeneration.from_pretrained('/kaggle/working/the_T5_model_rds').to(device)
    # Tải mô hình đã được huấn luyện trước đó từ đường dẫn chỉ định và chuyển mô hình sang thiết bị (CPU/GPU)
    
    model.eval()
    # Đặt mô hình vào chế độ đánh giá (evaluation mode) để tắt các kỹ thuật không cần thiết (như dropout)

    testion_dataloader = DataLoader(CustomDataset(data_test), shuffle=False, batch_size=batch_size)
    # Tạo DataLoader cho tập dữ liệu kiểm tra (test set) mà không xáo trộn (shuffle) và sử dụng kích thước batch được định nghĩa
    
    rougeL = evaluate_rougeL(model, testion_dataloader)
    # Đánh giá mô hình bằng cách tính điểm ROUGE-L trên tập kiểm tra
    
    print(f"Test completed, Testion ROUGE-L : {rougeL:.4f}")
    # Hiển thị kết quả điểm ROUGE-L sau khi kiểm tra


# Training model

In [19]:
## Chuẩn bị data
data_train = pd.read_csv("/kaggle/working/train.csv").dropna().to_dict(orient='records')[:10000]
data_val = pd.read_csv("/kaggle/working/val.csv").dropna().to_dict(orient='records')[:1000]
data_test = pd.read_csv("/kaggle/working/test.csv").dropna().to_dict(orient='records')[0:100]

In [20]:
# Đào tạo mô hình
Trainer()

Epochs:   0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 0/1250 [00:00<?, ?it/s]

Passing a tuple of `past_key_values` is deprecated and will be removed in Transformers v4.48.0. You should pass an instance of `EncoderDecoderCache` instead, e.g. `past_key_values=EncoderDecoderCache.from_legacy_cache(past_key_values)`.


Epoch: 1, Loss: 0.5668775238454342


  0%|          | 0/125 [00:00<?, ?it/s]

Epoch 1 completed. Validation ROUGE-L: 0.2786
Model saved at epoch 1 with ROUGE-L: 0.2786
Training completed. Best ROUGE-L score: 0.2786


In [21]:
# Đánh giá mô hình
Tester()

  0%|          | 0/13 [00:00<?, ?it/s]

Test completed, Testion ROUGE-L : 0.3301


# Predict for some sample

In [22]:
def predict(model, tokenizer, text, device):
    # Hàm dự đoán (predict) sử dụng mô hình đã huấn luyện để sinh ra văn bản đầu ra từ đầu vào

    model.eval()
    # Đặt mô hình vào chế độ đánh giá (evaluation mode) để tắt dropout và các kỹ thuật không cần thiết

    # Tokenize input text
    inputs = tokenizer(text, return_tensors="pt", padding=True, truncation=True, max_length=512).to(device)['input_ids']
    # Tokenize văn bản đầu vào:
    # - return_tensors="pt": trả về kết quả dưới dạng tensor của PyTorch
    # - padding=True: thêm padding để đảm bảo chuỗi đầu vào có độ dài đồng nhất
    # - truncation=True: cắt bớt chuỗi nếu vượt quá độ dài tối đa (max_length)
    # - max_length=512: độ dài tối đa của chuỗi đầu vào
    # Kết quả được chuyển sang thiết bị (CPU hoặc GPU)

    # Sinh ra dự đoán từ mô hình
    with torch.no_grad():
        # Tắt tính toán gradient để tiết kiệm bộ nhớ và tăng tốc trong giai đoạn đánh giá

        outputs = model.generate(input_ids=inputs, max_length=max_output_length, num_beams=num_beams, length_penalty=length_penalty)
        # Sử dụng phương pháp generate của mô hình để sinh chuỗi đầu ra:
        # - input_ids: chuỗi đầu vào đã được tokenize
        # - max_length: độ dài tối đa của chuỗi đầu ra
        # - num_beams: số lượng beam search để mở rộng không gian tìm kiếm chuỗi tốt nhất
        # - length_penalty: hệ số phạt độ dài để kiểm soát độ dài chuỗi đầu ra

    # Chuyển output từ tensor thành text
    predicted_text = tokenizer.decode(outputs[0], skip_special_tokens=True)
    # Giải mã tensor đầu ra (`outputs`) thành chuỗi văn bản:
    # - skip_special_tokens=True: bỏ qua các token đặc biệt (như [CLS], [SEP], <pad>, v.v.)
    
    return predicted_text
    # Trả về chuỗi văn bản đã được sinh bởi mô hình


In [23]:
text = """How to parse JSON in <mask> <body> I'm trying to parse a JSON string in Python, which contains nested dictionaries and lists. I want to extract specific values from it. What library or method should I use? <code> import json\n data = '{"name": "Alice", "age": 25}'\n parsed = json.loads(data)\n print(parsed["name"])"""
# How to parse JSON in Python


In [24]:
print(predict(model, tokenizer, text, device))

Python string with nested dictionaries and lists?


In [25]:
text = """Why does my thread <mask> <body> I'm working on a multithreaded Java application. I spawn several threads to handle incoming requests, but sometimes the application freezes or exits unexpectedly without any error message. I suspect it's related to thread lifecycle or synchronization. <code> public class Worker extends Thread { public void run() { while (running) { process(); } } } \n Worker w = new Worker(); \n w.start(); \n w.stop();"""
# Why does my thread terminate unexpectedly in Java

In [26]:
print(predict(model, tokenizer, text, device))

freeze or exit unexpectedly without any error message?
